In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Data Preprocessing and feature extraction
- Paper uses time-frequency patches that are extracted from the log-scaled mel-spectrogram as input into the model. (Details in “Unsupervised feature learning for urban sound classification" paper)

## 1.1 Reading in data using librosa

## 1.2 Extracting log-scaled mel spectrogram using Essentia

## 1.3 Getting the TF patches

# 2. Baseline model architecture
- 3 Convolutional Layers
- 2 Dense Linear Layers

In [2]:
# Code heavily references the paper on Environmental Sound Classification on Microcontrollers using Convolutional Neural Networks
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2

In [ ]:
# To check for how many channels we have

frames=128 # Number of time frames we have
bands=128 # Number of Mel Bands
channels=1# Number of input channels, determines what kind of convolution we use
num_labels=10 # Number of audio target labels
conv_size=(5,5) # Size of each filter/kernel
conv_block='conv'
downsample_size=(4,2) # Stride size
fully_connected=64 # Dense layer number of neurons
n_stages=None
n_blocks_per_stage=None
filters=24
kernels_growth=2 # After each convolutional layer, the paper includes growth of filter
dropout=0.5
use_strides=False

Conv2 = SeparableConv2D if conv_block == 'depthwise_separable' else Convolution2D


kernel = conv_size # Size of each filter/kernel
if use_strides: # How much each filter moves
  strides = downsample_size
  pool = (1, 1)
else:
  strides = (1, 1)
  pool = downsample_size

# First Block assumes 2D Convolution i.e. input has one channel

block1 = [
  Convolution2D(filters, kernel, padding='same', strides=strides,
  input_shape=(bands, frames, channels)),
  BatchNormalization(),
  MaxPooling2D(pool_size=pool),
  Activation('relu'),
  ]


block2 = [
  Conv2(filters*kernels_growth, kernel, padding='same', strides=strides),
  BatchNormalization(),
           MaxPooling2D(pool_size=pool),
  Activation('relu'),
]

block3 = [
  Conv2(filters*kernels_growth, kernel, padding='valid', strides=strides),
  BatchNormalization(),
  Activation('relu'),
]

backend = [
  Flatten(),
  Dropout(dropout),
  Dense(fully_connected, kernel_regularizer=l2(0.001)),
  Activation('relu'),
  Dropout(dropout),
  Dense(num_labels, kernel_regularizer=l2(0.001)),
  Activation('softmax'),
]
layers = block1 + block2 + block3 + backend
model = Sequential(layers)

# Training the model

# Testing the model